
Lab III: 機率模型
================
此 lab 中，我們將會透過 `tensorflow-probability`（TFP）此套件，學習以下的主題。

1. 認識 TFP 的分配（distribution）物件。

2. 利用變項之分配，搭配自動微分獲與優化器獲得最大概似估計值。

3. 了解如何將個別變項之分配物件組成聯合分配，建立複雜的機率模型。

4. 應用前述之知識，建立一可進行廣義線性模型分析之類型（class）。

TFP 之安裝與基礎教學，可參考 [TFP官方網頁](https://www.tensorflow.org/probability/install)。在安裝完成後，可透過以下的指令載入其與 `tensorflow`

In [1]:
import tensorflow_probability as tfp
import tensorflow as tf

## 分配物件
TFP 最為核心的物件為分配物件，其用於表徵一機率分配。TFP 涵蓋了許多不同的機率分配，其名單可至 [官方網頁](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions) 查看。

在實務上，我們常將 TFP 的分配模組儲存為 `tfd`，以便於使用，即：

In [2]:
tfd = tfp.distributions

### 分配物件之基本操作

以常態分配為例，我們可透過以下程式碼產生一表徵常態分配之物件，其平均數為0，標準差為1（尾端的小數點表示浮點數，而非整數）：

In [3]:
normal = tfd.Normal(loc=0., scale=1.)

透過此分配物件，我們可以產生對應之隨機樣本

In [4]:
print("random sample with shape ():\n",
      normal.sample())
print("random sample with shape (3,):\n",
      normal.sample(sample_shape=3))
print("random sample with shape (2,3):\n",
      normal.sample(sample_shape=(2, 3)))

random sample with shape ():
 tf.Tensor(2.2713256, shape=(), dtype=float32)
random sample with shape (3,):
 tf.Tensor([ 2.3771358  -1.8249989  -0.37349173], shape=(3,), dtype=float32)
random sample with shape (2,3):
 tf.Tensor(
[[ 1.7267758   0.23061198 -1.9052708 ]
 [ 0.26800588 -0.07182376  0.38082814]], shape=(2, 3), dtype=float32)


我們亦可給定實現值來評估其在該分配下之累積機率值：

In [5]:
print("cumulative probability given value with shape ():\n",
      normal.cdf(value=0), "\n")
print("cumulative probability given value with (3,):\n",
      normal.cdf(value=[-1, 0, .5]), "\n")
print("cumulative probability given value with (2,3):\n",
      normal.cdf(value=[[-1, 0, .5], [-2, 1, 3]]))


cumulative probability given value with shape ():
 tf.Tensor(0.5, shape=(), dtype=float32) 

cumulative probability given value with (3,):
 tf.Tensor([0.15865526 0.5        0.69146246], shape=(3,), dtype=float32) 

cumulative probability given value with (2,3):
 tf.Tensor(
[[0.15865526 0.5        0.69146246]
 [0.02275013 0.8413447  0.9986501 ]], shape=(2, 3), dtype=float32)


或是對數機率值：

In [6]:
print("log-probability given value with shape ():\n",
      normal.log_prob(value=0), "\n")
print("log-probability given value with (3,):\n",
      normal.log_prob(value=[-1, 0, .5]), "\n")
print("log-probability given value with (2,3):\n",
      normal.log_prob(value=[[-1, 0, .5], [-2, 1, 3]]))

log-probability given value with shape ():
 tf.Tensor(-0.9189385, shape=(), dtype=float32) 

log-probability given value with (3,):
 tf.Tensor([-1.4189385 -0.9189385 -1.0439385], shape=(3,), dtype=float32) 

log-probability given value with (2,3):
 tf.Tensor(
[[-1.4189385 -0.9189385 -1.0439385]
 [-2.9189386 -1.4189385 -5.4189386]], shape=(2, 3), dtype=float32)


### 分配物件之形狀
分配物件的形狀比起張量的形狀較為複雜些，其產生的樣本共牽涉到三種形狀：

1. 樣本形狀（sample shape），為在使用分配物件產生樣本時之形狀（即為前一小節使用 `.sample()` 時所設定的 `sample_shape`），其產生的資料彼此間為獨立且相同分配的（independent and identically distributed）。

2. 批次形狀（batch shape），為建立分配物件時所設定的形狀（其透過參數的形狀決定），其可用於產生批次的樣本，批次樣本間彼此獨立，但其邊際分配之參數可以不同。

3. 事件形狀（event shape），即多變量分配之變數形狀，如 $P$ 維多元常態分配的形狀即為 `(P,)`，在同一事件下產生的資料其變數間可為相依，且各邊際分配之參數也未必相同。

而分配物件的形狀則牽涉到批次與事件兩種，可透過直接列印分配物件查看

In [7]:
print(normal)

tfp.distributions.Normal("Normal", batch_shape=[], event_shape=[], dtype=float32)


或是使用 `.batch_shape` 與 `.event_shape` 獲得：

In [8]:
print(normal.batch_shape)
print(normal.event_shape)

()
()


由於之前所創立的 `normal` 其用於產生純量之常態分配隨機變數，故 `.batch_shape` 與 `.event_shape` 兩者皆為 `()`。

批次形狀之設定，乃經由對分配參數形狀之推論獲得，如

In [9]:
normal_batch = tfd.Normal(loc=[0., 1.], scale=[1., .5])
print(normal_batch)

tfp.distributions.Normal("Normal", batch_shape=[2], event_shape=[], dtype=float32)


前述分配的 `batch_shape` 為 `(2,)`，其可用於產生一組兩個來自常態分配之變數，其中一個平均數為0，變異數為1，另一個平均數為1，變異數為.5，如

In [10]:
print("random sample with sample_shape ():\n",
      normal_batch.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      normal_batch.sample(sample_shape=3), "\n")
print("random sample with sample_shape (2,3):\n",
      normal_batch.sample(sample_shape=(2,3)))

random sample with sample_shape ():
 tf.Tensor([1.5389309 1.058266 ], shape=(2,), dtype=float32) 

random sample with sample_shape (3,):
 tf.Tensor(
[[ 0.8262944  -0.20904672]
 [-0.41877052  0.37379056]
 [ 1.1131867   1.7374542 ]], shape=(3, 2), dtype=float32) 

random sample with sample_shape (2,3):
 tf.Tensor(
[[[ 0.41234437  1.1895566 ]
  [ 0.21679199  0.37657076]
  [ 0.7545709   0.76228845]]

 [[ 0.1450507   1.1977061 ]
  [-2.324742    0.86529374]
  [-0.02887968  1.0968765 ]]], shape=(2, 3, 2), dtype=float32)


我們亦可使用所創立的 `normal_batch` 來度量輸入數值的對數機率值：

In [11]:
print("log-probability given value with shape ():\n",
      normal_batch.log_prob(0), "\n")
print("log-probability given value with shape (2,):\n",
      normal_batch.log_prob([0, 0]), "\n")
print("log-probability given value with shape (2,1):\n",
      normal_batch.log_prob([[0], [0]]))

log-probability given value with shape ():
 tf.Tensor([-0.9189385 -2.2257915], shape=(2,), dtype=float32) 

log-probability given value with shape (2,):
 tf.Tensor([-0.9189385 -2.2257915], shape=(2,), dtype=float32) 

log-probability given value with shape (2,1):
 tf.Tensor(
[[-0.9189385 -2.2257915]
 [-0.9189385 -2.2257915]], shape=(2, 2), dtype=float32)


這邊我們可以觀察到，前兩者種寫法獲得一樣的數值，皆表示 `[0, 0]` 此向量於 `normal_batch` 下的對數機率。對第一種寫法來說，其輸入為純量，因此，使用到了廣播（broadcasting）的概念，將0的數值轉為 `[0,0]` 後再進行評估，第二種寫法則是較為標準，其直接輸入了 `[0,0]`，與 `normal_batch` 的 `batch_size` 相同。而第三種寫法，可以想成輸入了一 `sample_shape` 為2的資料，而每筆觀測值的0皆會透過廣播拓展為 `[0, 0]`，故回傳了每筆觀測值於 `normal_batch` 之對數機率。


事件形狀僅適用於多變量之分配，以多元常態（multivariate normal）分配為例，其需給定一平均數向量與共變異數矩陣（在這邊，我們採用的 `tfd.MultivariateNormalTriL` 需給定的是共變異數矩陣的 Cholesky 分解）：

In [12]:
mvn = tfd.MultivariateNormalTriL(
    loc=[0, 1],
    scale_tril=tf.linalg.cholesky([[1., 0.], [0., .5]]))
print(mvn)

tfp.distributions.MultivariateNormalTriL("MultivariateNormalTriL", batch_shape=[], event_shape=[2], dtype=float32)


我們可看到此分配物件的 `event_shape` 是 `(2)`，與此多元常態分配的維度相同，其可用於產生服從多元常態分配之資料

In [13]:
print("random sample with sample_shape ():\n",
      mvn.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      mvn.sample(sample_shape=3), "\n")
print("random sample with sample_shape (2, 3):\n",
      mvn.sample(sample_shape=(2, 3)))

random sample with sample_shape ():
 tf.Tensor([0.2162181  0.46002054], shape=(2,), dtype=float32) 

random sample with sample_shape (3,):
 tf.Tensor(
[[-0.25399727  1.3331991 ]
 [ 0.50582093  0.54873157]
 [-0.08385251  0.15000951]], shape=(3, 2), dtype=float32) 

random sample with sample_shape (2, 3):
 tf.Tensor(
[[[-0.07636248  0.4365208 ]
  [-0.9659546  -0.01564229]
  [ 1.0694548   2.2594252 ]]

 [[-0.08555713  1.887286  ]
  [-0.17217642  2.4205818 ]
  [-0.3970989   1.2988566 ]]], shape=(2, 3, 2), dtype=float32)


同樣的，該物件亦可用於評估給定數值下的對數機率：

In [14]:
print("log-probability given value with shape (2,):\n",
      mvn.log_prob([0, 0]), "\n")
print("log-probability given value with shape (2,1):\n",
      mvn.log_prob([[0, 0], [0, 0]]))

log-probability given value with shape (2,):
 tf.Tensor(-2.4913032, shape=(), dtype=float32) 

log-probability given value with shape (2,1):
 tf.Tensor([-2.4913032 -2.4913032], shape=(2,), dtype=float32)


儘管此 `mvn` 表徵的二維之多元常態分配，其平均數與共變異數矩陣之設定，與先前 `normal_batch`是等價的，但在使用 `mvn` 評估機率時，需注意：（1）先前針對 `batch_shape` 此面向的廣播，不再適用於 `event_shape`，故 `mvn.log_prob(0)` 會產生錯誤訊息；（2）針對每筆觀測值，其計算的是在此多元常態分配下的聯合機率，因此，只會獲得一個對數機率值。


分配物件的 `batch_shape` 可透過 `tfd.Independent` 此物件轉為 `event_shape`，如

In [15]:
tfd.Independent(normal_batch, reinterpreted_batch_ndims=1)

<tfp.distributions.Independent 'IndependentNormal' batch_shape=[] event_shape=[2] dtype=float32>

在多變量的分配之下，前述介紹的批次形狀與事件形狀，可以合併使用：

In [16]:
mvn_batch = tfd.MultivariateNormalTriL(
    loc=[[0, 1],
         [1, 2],
         [2, 3]],
    scale_tril=tf.linalg.cholesky([[1., 0.], [0., .5]]))
mvn_batch

<tfp.distributions.MultivariateNormalTriL 'MultivariateNormalTriL' batch_shape=[3] event_shape=[2] dtype=float32>

這裡，其 `batch_shape` 為 `(3)`，值得注意的是，這邊我們僅設定了一個共變異數矩陣，因此，其會透過廣播的機制，與三個平均數向量做對應。同樣的，我們可以用此 `mvn_batch` 來產生樣本資料


In [17]:
print("random sample with sample_shape ():\n",
      mvn_batch.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      mvn_batch.sample(sample_shape=3), "\n")
print("random sample with sample_shape (2, 3):\n",
      mvn_batch.sample(sample_shape=(2, 3)))

random sample with sample_shape ():
 tf.Tensor(
[[-0.8442798   1.6098676 ]
 [ 0.05365855  1.8673433 ]
 [ 1.2360637   3.1306837 ]], shape=(3, 2), dtype=float32) 

random sample with sample_shape (3,):
 tf.Tensor(
[[[-1.8156862   0.6523211 ]
  [ 1.7010512  -0.16461039]
  [ 0.34791625  1.9542217 ]]

 [[-1.0917486   0.5257746 ]
  [ 2.0450764   2.7479446 ]
  [ 0.6193794   2.1641443 ]]

 [[ 1.6523548   0.2863677 ]
  [ 0.19571602  3.248878  ]
  [ 2.320044    1.8242878 ]]], shape=(3, 3, 2), dtype=float32) 

random sample with sample_shape (2, 3):
 tf.Tensor(
[[[[-0.5093707   0.81250143]
   [ 1.9468127   2.9683397 ]
   [ 1.3619013   2.2389302 ]]

  [[-0.26521453  0.95227057]
   [ 3.1439462   1.5873938 ]
   [ 1.4592228   1.5732392 ]]

  [[-1.6538379   1.6909251 ]
   [ 1.2586976   1.7571654 ]
   [ 2.8081112   3.2831683 ]]]


 [[[-0.30845714  0.38611084]
   [ 0.34927046  1.7144573 ]
   [ 0.6021681   3.6861699 ]]

  [[ 0.27426484  1.322668  ]
   [-0.09283876  2.9660397 ]
   [ 2.4892762   2.6491864 

## 最大概似估計法



### 可學習的分配與求解
使用 `tensorflow` 來進行最大概似法，有許多種做法，其中，最重要的關鍵就在於如何建構概似函數。事實上，在前一小節中，我們已經可以計算在給定參數下，某個隨機樣本實現值之可能性，因此，關鍵就在於如何讓前述之可能性，轉為參數數值之函數，而最簡單的做法，就是將參數設為 `tf.Variable`，搭配自動微分與優化器對其進行更新。

以下的程式碼建立了一 `batch_size` 為2的常態分配模型，我們可透過 `.trainable_varialbes` 來查看哪些變數是可以透過訓練更新其數值的。

In [18]:
batch_size = 2
normal_model = tfd.Normal(
        loc=tf.Variable(tf.zeros(batch_size), name='loc'),
        scale=tf.Variable(tf.ones(batch_size), name='scale'))
print("normal model:\n", normal_model, "\n")
print("parameters in normal model:\n", normal_model.trainable_variables)

normal model:
 tfp.distributions.Normal("Normal", batch_shape=[2], event_shape=[], dtype=float32) 

parameters in normal model:
 (<tf.Variable 'loc:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'scale:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>)


TFP 的官方教學中，將前述的分配稱作可學習的分配（learnable distribution）。接著，我們在目前給定的參數數值下，產生一隨機樣本，此樣本在目前參數數值下的可能性，可簡單地使用 `.log_prob()` 方法與加總平均的計算獲得：

In [19]:
sample_size = 1000
x = normal_model.sample(sample_shape=sample_size)
loss_value = -tf.reduce_mean(tf.reduce_sum(normal_model.log_prob(x), axis = 1))
print("negative likelihood value is ", loss_value.numpy())

negative likelihood value is  2.8390605


最後，我們就可以透過優化器來求最大概似解了。在這邊需特別注意的是，由於 `loc` 與 `scale` 原本的數值為真實參數的數值，為了要展示優化器的正確運作，我們將其起始值設為一個較差的數值。

In [20]:
epochs = 400
tol = 10**(-5)
learning_rate = 1.0
normal_model.loc.assign([1., 1.])
normal_model.scale.assign([.5, .5])
optimizer = tf.optimizers.Adam(learning_rate)
for epoch in tf.range(epochs):
    with tf.GradientTape() as tape:
        loss_value = -tf.reduce_mean(tf.reduce_sum(normal_model.log_prob(x), axis = 1))
    gradients = tape.gradient(loss_value,
                              normal_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,
                                  normal_model.trainable_variables))
    if (tf.reduce_max(
            [tf.reduce_mean(
                tf.math.abs(x)) for x in gradients]).numpy()) < tol:
        print("{n} Optimizer Converges After {i} Iterations".format(
            n=optimizer.__class__.__name__, i=epoch))
        break

Adam Optimizer Converges After 276 Iterations


接著，我們比較優化器求得的解，以及分析解之間的差異（常態分配平均數與標準差的分析解即為樣本平均數與除上 $N$ 的標準差）

In [21]:
print("ML mean estimate: \n", 
      normal_model.loc.numpy())
print("ML standard deviation estimate: \n", 
      normal_model.scale.numpy())

ML mean estimate: 
 [-0.03601326 -0.03063894]
ML standard deviation estimate: 
 [0.99562484 1.0044298 ]


In [22]:
print("sample mean: \n", 
      tf.reduce_mean(x, axis = 0).numpy())
print("sample standard deviation: \n", 
      tfp.stats.stddev(x, sample_axis = 0).numpy())

sample mean: 
 [-0.0360147  -0.03063884]
sample standard deviation: 
 [0.9956158 1.0044307]


我們可看到兩組解的數值幾乎相同，顯示優化器在這邊有確實地找到最大概似解。

由於前述的優化程式碼，在後續的小節中仍然會持續被用到，故我們將其包成一個 `train` 函數，以便後續使用。

In [23]:
def train(model, x, optimizer, initial_values, epochs, tol):
    for initial_value, trainable_variable in \
            zip(initial_values, model.trainable_variables):
        trainable_variable.assign(initial_value)
    for epoch in tf.range(epochs):
        with tf.GradientTape() as tape:
            loss_value = -tf.reduce_mean(
                tf.reduce_sum(model.log_prob(x), axis = 1))
        gradients = tape.gradient(loss_value,
                                  model.trainable_variables)
        optimizer.apply_gradients(zip(gradients,
                                      model.trainable_variables))
        if (tf.reduce_max(
                [tf.reduce_mean(
                    tf.math.abs(x)) for x in gradients]).numpy()) < tol:
            print("{n} Optimizer Converges After {i} Iterations".format(
                n=optimizer.__class__.__name__, i=epoch))
            break

### 利用對射進行變數轉換
前述的最大概似估計過程，並未對於參數估計之數值進行限制，其考慮的是非限制的優化問題（unconstrained optimization problem），然而，在實際進行優化時，若未對於參數數值進行限制的話，可能會獲得不合理之估計值（如負的變異數等）。

在 TFP 的架構中，主要是透過對於參數進行對射（bijection），將原始受限制的參數轉為非限制的參數後進行估計。TFP 所內建的對射函數，可參考其 [官方網頁](https://www.tensorflow.org/probability/api_docs/python/tfp/bijectors)。

在下面的範例中，我們利用 `tfp.util.TransformedVariable` 與 `tfb.Exp()` ，將常態分配的變異數 $\sigma$ 參數化為 $\exp(\gamma)$，將原本需進行估計 $\mu$ 與 $\sigma$ 的優化問題，轉為估計 $\mu$ 與 $\gamma$，此時，我們不需要對 $\gamma$ 的數值範圍進行限制，其在透過 $\exp$ 函數的轉換後，會自動符合模型隱含的限制式。

In [24]:
tfb = tfp.bijectors
batch_size = 2
normal_model_tr = tfd.Normal(
    loc=tf.Variable(tf.zeros(batch_size), name='loc'),
    scale=tfp.util.TransformedVariable(
        tf.ones(batch_size),
        bijector=tfb.Exp(), name="scale"))
print("normal model:\n", normal_model_tr, "\n")
print("parameters in normal model:\n", normal_model_tr.trainable_variables)

normal model:
 tfp.distributions.Normal("Normal", batch_shape=[2], event_shape=[], dtype=float32) 

parameters in normal model:
 (<tf.Variable 'loc:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'scale:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>)


我們可以直接使用前述的優化程式碼來獲得重新參數化後的參數估計

In [25]:
train(model=normal_model_tr,
      x=x,
      optimizer=tf.optimizers.Adam(learning_rate=1.),
      initial_values=([1., 1.], [.5, .5]),
      epochs=400,
      tol=10**(-5))

print("ML mean estimate: \n",
      normal_model_tr.loc.numpy())
print("ML standard deviation estimate: \n",
      normal_model_tr.scale.numpy())

Adam Optimizer Converges After 206 Iterations
ML mean estimate: 
 [-0.03603007 -0.03065409]
ML standard deviation estimate: 
 [0.99563926 1.0044488 ]


此變數轉換的作法，在進行多元常態分配共變異數估計時，特別的有幫助。一般來說，多元常態分配假設共變異數矩陣為對稱的正定矩陣（positive definite matrix），然而，一般在進行參數估計時，並無法保證此條件被滿足

即便 `tfd.MultivariateNormalTriL` 已採用了

In [26]:
loc = tf.constant([0., 1., -1.])
cov = tf.constant([[1, .3, .6], [.3, .5, .1], [.6, .1, 1.5]])
to_scale_tril = tfb.Chain([
    tfb.Invert(
        tfb.FillTriangular(validate_args=True)),
    tfb.TransformDiagonal(
        tfb.Invert(tfb.Exp(validate_args=True)))])

mvn_model = tfd.MultivariateNormalTriL(
    loc=tf.Variable(loc, name='loc'),
    scale_tril=tfp.util.TransformedVariable(
        tf.linalg.cholesky(cov).numpy(),
        bijector=tfp.bijectors.FillScaleTriL(
            diag_bijector = tfb.Exp()),
        name="scale"))
print(mvn_model)
print(mvn_model.trainable_variables)

tfp.distributions.MultivariateNormalTriL("MultivariateNormalTriL", batch_shape=[], event_shape=[3], dtype=float32)
(<tf.Variable 'loc:0' shape=(3,) dtype=float32, numpy=array([ 0.,  1., -1.], dtype=float32)>, <tf.Variable 'scale:0' shape=(6,) dtype=float32, numpy=
array([ 5.8611017e-02, -1.2493902e-01,  6.0000002e-01, -1.0013630e-05,
       -4.4581467e-01,  3.0000001e-01], dtype=float32)>)


In [27]:
print(tf.linalg.cholesky(cov).numpy())
print(mvn_model.scale_tril.numpy())
print(mvn_model.covariance().numpy())


[[ 1.          0.          0.        ]
 [ 0.3         0.64031243  0.        ]
 [ 0.6        -0.12493902  1.0603727 ]]
[[ 1.          0.          0.        ]
 [ 0.3         0.64031243  0.        ]
 [ 0.6        -0.12493902  1.0603727 ]]
[[1.        0.3       0.6      ]
 [0.3       0.5       0.1      ]
 [0.6       0.1       1.5000001]]
